In [262]:
import pandas as pd
import numpy as np

In [263]:
#set up base dataframe
df = pd.read_csv('/content/drive/MyDrive/ml-project/data/salaries.csv')
df.head()

,playerName,seasonStartYear,salary,inflationAdjSalary
0,Michael Jordan,1996,"$30,140,000","$52,258,566"
1,Horace Grant,1996,"$14,857,000","$25,759,971"
2,Reggie Miller,1996,"$11,250,000","$19,505,934"
3,Shaquille O'Neal,1996,"$10,714,000","$18,576,585"
4,Gary Payton,1996,"$10,212,000","$17,706,187"


In [264]:
#get player profile info
player_info = pd.read_csv('/content/drive/MyDrive/ml-project/data/player_info.csv')
player_info.head()

,playerName,From,To,Pos,Ht,Wt,birthDate,Colleges
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State
2,Kareem Abdul-Jabbar*,1970,1989,C,7-2,225.0,"April 16, 1947",UCLA
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",LSU
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974","Michigan, San Jose State"


In [265]:
#replace * and convert birthday to date time
player_info["playerName"] = player_info["playerName"].str.replace("*","")
player_info['birthDate'] = pd.to_datetime(player_info['birthDate']).dt.date
player_info.head()

<ipython-input-265-0c69eee07ba8>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  player_info["playerName"] = player_info["playerName"].str.replace("*","")


,playerName,From,To,Pos,Ht,Wt,birthDate,Colleges
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,1968-06-24,Duke
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,1946-04-07,Iowa State
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,1947-04-16,UCLA
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,1969-03-09,LSU
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,1974-11-03,"Michigan, San Jose State"


In [266]:
player_info.dtypes

playerName     object
From            int64
To              int64
Pos            object
Ht             object
Wt            float64
birthDate      object
Colleges       object
dtype: object

In [267]:
#figure out who is a duplicate name and start season
dup = player_info[player_info[['playerName','From']].duplicated(keep=False)]
dupNames = ["Bill Bradley","Tony Mitchell"]
dup

,playerName,From,To,Pos,Ht,Wt,birthDate,Colleges
469,Bill Bradley,1968,1968,G,5-11,165.0,1941-06-16,NaN
470,Bill Bradley,1968,1977,F-G,6-5,205.0,1943-07-28,Princeton
3074,Tony Mitchell,2014,2014,F,6-6,216.0,1989-08-07,Alabama
3075,Tony Mitchell,2014,2014,F,6-8,235.0,1992-04-07,University of North Texas


In [268]:
#create birthday
df["birthDate"] = np.nan
df["startYear"] = np.nan
df["position"] = np.nan
df["height"] = np.nan
df["weight"] = np.nan
for index, row in player_info.iterrows():
  name = row['playerName']
  if name not in dupNames:
    birth = player_info.loc[player_info['playerName'] == name]['birthDate'].values[0]
    df.loc[df["playerName"] == name, "birthDate"] = birth

    start = player_info.loc[player_info['playerName'] == name]['From'].values[0]
    df.loc[df["playerName"] == name, "startYear"] = start

    position = player_info.loc[player_info['playerName'] == name]['Pos'].values[0]
    df.loc[df["playerName"] == name, "position"] = position

    height = player_info.loc[player_info['playerName'] == name]['Ht'].values[0]
    df.loc[df["playerName"] == name, "height"] = height

    weight = player_info.loc[player_info['playerName'] == name]['Wt'].values[0]
    df.loc[df["playerName"] == name, "weight"] = weight
df.head()

,playerName,seasonStartYear,salary,inflationAdjSalary,birthDate,startYear,position,height,weight
0,Michael Jordan,1996,"$30,140,000","$52,258,566",1963-02-17,1985.0,G-F,6-6,198.0
1,Horace Grant,1996,"$14,857,000","$25,759,971",1965-07-04,1988.0,F-C,6-10,215.0
2,Reggie Miller,1996,"$11,250,000","$19,505,934",1965-08-24,1988.0,G-F,6-7,185.0
3,Shaquille O'Neal,1996,"$10,714,000","$18,576,585",1972-03-06,1993.0,C,7-1,325.0
4,Gary Payton,1996,"$10,212,000","$17,706,187",1968-07-23,1991.0,G,6-4,180.0


In [269]:
name = 'Michael Jordan'
rows = df.loc[df['playerName'] == name]
rows

,playerName,seasonStartYear,salary,inflationAdjSalary,birthDate,startYear,position,height,weight
0,Michael Jordan,1996,"$30,140,000","$52,258,566",1963-02-17,1985.0,G-F,6-6,198.0
415,Michael Jordan,1997,"$33,140,000","$56,169,715",1963-02-17,1985.0,G-F,6-6,198.0
2573,Michael Jordan,2001,"$1,000,000","$1,526,382",1963-02-17,1985.0,G-F,6-6,198.0
3023,Michael Jordan,2002,"$1,030,000","$1,555,569",1963-02-17,1985.0,G-F,6-6,198.0


In [270]:
df.isnull().sum()

playerName               0
seasonStartYear          0
salary                   0
inflationAdjSalary       0
birthDate             1075
startYear             1075
position              1075
height                1075
weight                1075
dtype: int64

In [271]:
df.count()

playerName            11583
seasonStartYear       11583
salary                11583
inflationAdjSalary    11583
birthDate             10508
startYear             10508
position              10508
height                10508
weight                10508
dtype: int64

In [274]:
df.to_csv('/content/drive/MyDrive/ml-project/data/cleaned.csv')